## Women Writing Africa, Data Mining Process
- Convert PDF to txt document

In [1]:
import numpy as np
import pandas as pd
import os

import re

from pathlib import Path

In [2]:
raw_book_path = Path("Datasets") / "Women-Writing-Africa-The-Southern-Region-by-M.-J.txt"
with open(raw_book_path, encoding="utf8") as f:
    lines = f.readlines()

In [3]:
# Remove "¬" symbols for line breaks
lines_temp = [lines[0]]
in_line_break = False
for line in lines[1:]:

    if in_line_break:
        lines_temp[-1] = lines_temp[-1].replace("¬\n", "") + line
    else:
        lines_temp.append(line)

    if "¬" in lines_temp[-1]:
        in_line_break = True
    else:
        in_line_break = False
lines = lines_temp

In [4]:
for key, line in enumerate(lines):
    if "¬" in line:
        print(line)
        print(key)

In [5]:
print(lines[0:10])

['\n', '\n', '\n', '\n', '\x0c          Anonymous, SONG OF THE AFFLICTED\n', '                                Lesotho 1842      Sesotho\n', '\n', '\n', ' Song ot the Atflicted   is a lament that tails within the nexus of traditional warfare\n', 'in Lesotho. Departure for battle involved strengthening rituals meant to discourage\n']


In [10]:
language_list = ["English", "Sesotho", "Dutch", "German", "isiXhosa", "KhoeKhoeGowab", "isiZulu",
                 "Xam", "otjiHerero", "Afrikaans", "Shangaani", "siSwati", "Setswana", "oshiNdonga"
                 "siNdebele", "siNdebele", "chiShona", "Ju/’hoansi", "isiXhosa", "oshiNdonga", "uuKwanyama"
                 "Tonga", "siSwati", "Nyemba"]

country_list = ["Lesotho", "South Africa", "Namibia", "Zimbabwe", "Botswana", "Swaziland"]

for i in range(len(lines)-2):
    if lines[i-1] == "\n" or "\x0c" in lines[i-1]:
        year_check_1 = re.search(r'\b(18[0-9][0-9]|19[0-9][0-9]|20[0-1][0-9])\b', lines[i+1])
        year_check_2 = re.search(r'\b(18[0-9][0-9]|19[0-9][0-9]|20[0-1][0-9])\b', lines[i+2])
        
        if year_check_1 or year_check_2:

            contains_country = []
            for country in country_list:
                if country in " ".join(lines[i:i+3]):
                    contains_country.append(True)
                else:
                    contains_country.append(False)

            if "\n" in lines[i+2:i+4] and any(contains_country):
                print(lines[i:i+3])

['\n', '\x0c          Anonymous, SONG OF THE AFFLICTED\n', '                                Lesotho 1842      Sesotho\n']
['\x0c          Anonymous, SONG OF THE AFFLICTED\n', '                                Lesotho 1842      Sesotho\n', '\n']
['\n', '               Kaatje Nieuwveldt, TESTIMONY\n', '                          South Africa 1858      Dutch\n']
['               Kaatje Nieuwveldt, TESTIMONY\n', '                          South Africa 1858      Dutch\n', '\n']
['                              Emma Sandile,\n', '            Letters and Land Submission\n', '                       South Africa 1860-1883        English\n']
['             Urieta (Johanna Maria) Kazahendike                        ,\n', '               God’s Peace and Blessing\n', '                             Namibia 1861      German\n']
['\n', '           Eliza M., Account of Cape Town\n', '                          South Africa 1863      isiXhosa\n']
['           Eliza M., Account of Cape Town\n', '              